## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-20-20-49-49 +0000,nyt,Where’s the Beef? Trump Hopes Argentina Can He...,https://www.nytimes.com/2025/10/20/us/politics...
1,2025-10-20-20-48-25 +0000,nyt,The Indonesian Free-Food Program That Has Sick...,https://www.nytimes.com/2025/10/18/world/asia/...
2,2025-10-20-20-48-24 +0000,nypost,NYC school named after Knicks legend Earl Monr...,https://nypost.com/2025/10/20/us-news/nyc-scho...
3,2025-10-20-20-48-19 +0000,bbc,Junior minister reveals breast cancer diagnosis,https://www.bbc.com/news/articles/c9wv5d7rv4lo...
4,2025-10-20-20-37-09 +0000,nypost,Woke Portland slapped with civil rights compla...,https://nypost.com/2025/10/20/us-news/woke-por...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2363/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
2,trump,29
58,new,17
228,louvre,14
65,china,14
48,court,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
20,2025-10-20-20-06-32 +0000,latimes,Trump again suggests sending troops to San Fra...,https://www.latimes.com/california/story/2025-...,73
5,2025-10-20-20-36-40 +0000,nyt,Appeals Court Ruling Allows Trump to Deploy Na...,https://www.nytimes.com/2025/10/20/us/appeals-...,63
217,2025-10-19-22-46-24 +0000,bbc,China will soon have a new Five Year Plan. Her...,https://www.bbc.com/news/articles/cjr045x425vo...,62
60,2025-10-20-18-03-33 +0000,nypost,Trump warns Hamas with chilling statement as p...,https://nypost.com/2025/10/20/us-news/trump-wa...,61
29,2025-10-20-19-28-34 +0000,nypost,Demolition begins on East Wing of White House ...,https://nypost.com/2025/10/20/us-news/demoliti...,61


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
20,73,2025-10-20-20-06-32 +0000,latimes,Trump again suggests sending troops to San Fra...,https://www.latimes.com/california/story/2025-...
217,62,2025-10-19-22-46-24 +0000,bbc,China will soon have a new Five Year Plan. Her...,https://www.bbc.com/news/articles/cjr045x425vo...
125,42,2025-10-20-12-48-54 +0000,bbc,Deadly Gaza flare-up tests Israel-Hamas ceasefire,https://www.bbc.com/news/articles/czxk8k4xlv1o...
101,40,2025-10-20-15-10-02 +0000,nypost,French authorities admit they ‘failed’ in Louv...,https://nypost.com/2025/10/20/world-news/louvr...
129,36,2025-10-20-12-15-35 +0000,startribune,Amazon cloud computing outage disrupts Snapcha...,https://www.startribune.com/amazon-cloud-compu...
157,35,2025-10-20-10-00-00 +0000,latimes,How military's 'safe' plan to fire munitions o...,https://www.latimes.com/california/story/2025-...
110,35,2025-10-20-13-55-15 +0000,latimes,Supreme Court will decide if 'habitual drug us...,https://www.latimes.com/politics/story/2025-10...
2,28,2025-10-20-20-48-24 +0000,nypost,NYC school named after Knicks legend Earl Monr...,https://nypost.com/2025/10/20/us-news/nyc-scho...
203,27,2025-10-20-00-38-24 +0000,nypost,Two dead after cargo plane skids off runway in...,https://nypost.com/2025/10/19/world-news/cargo...
25,26,2025-10-20-19-37-30 +0000,nypost,Trump tears into Australian ambassador who bad...,https://nypost.com/2025/10/20/us-news/trump-te...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
